In [16]:
from datetime import datetime

from models import FineTuner
from generate_metrics import Metric

In [17]:
(datetime.now()).strftime("%Y-%m-%d")

'2023-12-22'

In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
model_checkpoint = "distilbert-base-uncased"
model_save_dir = f"/home/bhatt/ishan/TUM_Thesis/data/models/{model_checkpoint}_"+(datetime.now()).strftime("%Y-%m-%d")
ft_model_save_dir = f"/home/bhatt/ishan/TUM_Thesis/data/models/{model_checkpoint}-finetuned_"+(datetime.now()).strftime("%Y-%m-%d")

In [47]:
ft_model_save_dir
dataset_name = 'imdb'
ft_model_save_dir = f"/home/bhatt/ishan/TUM_Thesis/data/models/{model_checkpoint}-finetuned-{dataset_name}_"+(datetime.now()).strftime("%Y-%m-%d")
ft_model_save_dir

'/home/bhatt/ishan/TUM_Thesis/data/models/distilbert-base-uncased-finetuned-imdb_2023-12-22'

In [55]:
ft = FineTuner(model_name=model_checkpoint,from_local=True, model_save_dir=ft_model_save_dir,local_model_path='/home/bhatt/ishan/TUM_Thesis/data/models/distilbert-base-uncased-finetuned-imdb_2023-12-20/checkpoint-1500',
               ft_ds = 'imdb', random_init=False)

Initializing Model
Model Initialized!


In [56]:
ft.model_dir, ft.model_tag

('/home/bhatt/ishan/TUM_Thesis/data/models/distilbert-base-uncased-finetuned-imdb_2023-12-20/checkpoint-1500',
 'distilbert-base-uncased_FT_imdb')

In [50]:
# ft.finetune_model(dataset_name = 'imdb')

In [57]:
metric = Metric(model_name = model_checkpoint, model = ft.getModel(), tokenizer = ft.getTokenizer(), model_tag= ft.model_tag, model_dir = ft.model_dir)

In [44]:
metric.get_metric(metric = 'crows-pairs', input_file = '/home/bhatt/ishan/TUM_Thesis/data/metrics_ds/crows-pairs/data/crows_pairs_anonymized.csv', output_file = '/home/bhatt/ishan/TUM_Thesis/data/results/cp_results.csv')

  0%|          | 1/1508 [00:00<09:50,  2.55it/s]/home/bhatt/ishan/TUM_Thesis/src/metrics/crows_pairs.py:266: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_score = pd.concat([df_score,pd.DataFrame({'sent_more': [sent_more],
100%|██████████| 1508/1508 [04:00<00:00,  6.26it/s]


Total examples: 1508
Metric score: 55.9
Stereotype score: 56.71
Anti-stereotype score: 51.38
Num. neutral: 1 0.07

Metrics Saved in  /home/bhatt/ishan/TUM_Thesis/data/results/master_results.xlsx


In [58]:
metric.get_metric(metric = 'stereoset', input_file = '/home/bhatt/ishan/TUM_Thesis/data/metrics_ds/stereoset/dev.json', output_file = 'stereoset_results.txt',
                   output_dir = '/home/bhatt/ishan/TUM_Thesis/data/results')

100%|██████████| 8048/8048 [00:31<00:00, 258.65it/s]


Metrics Saved in  /home/bhatt/ishan/TUM_Thesis/data/results/master_results.xlsx
